In [111]:
import os

import cv2 as cv
import numpy as np
from ultralytics import YOLO

model_path = os.path.join(os.getcwd(), "YOLO_model", "weights", "best.pt")
model = YOLO(model_path)

In [112]:
piece_names = {
    "white-king": "K",
    "white-queen": "Q",
    "white-rook": "R",
    "white-bishop": "B",
    "white-knight": "N",
    "white-pawn": "P",
    "black-king": "k",
    "black-queen": "q",
    "black-rook": "r",
    "black-bishop": "b",
    "black-knight": "n",
    "black-pawn": "p",
}

In [113]:
def find_pieces(board):
    pos = np.zeros((8, 8), dtype=np.dtype("U1"))
    res = model.predict(board)[0]
    coords = res.boxes.xyxy.numpy().astype(int)[:, 0:2]
    labels = [model.names[int(c)] for c in res.boxes.cls]
    board_height, board_width, _ = board.shape
    print(labels)
    print(res.boxes.conf)
    for i in range(len(coords)):
        x, y = coords[i]
        posX = round(8 * x / board_width)
        posY = round(8 * y / board_height)
        pos[posY, posX] = piece_names[labels[i]]

    return pos

In [114]:
img = cv.imread("scr2.png")[:, :, :3]
res = find_pieces(img)
from analysis import get_fen

print(get_fen(res, "w"))
print(res)


0: 640x640 2 black-bishops, 1 black-king, 1 black-knight, 2 black-pawns, 1 black-queen, 2 black-rooks, 1 white-bishop, 1 white-king, 2 white-knights, 3 white-pawns, 1 white-queen, 1 white-rook, 167.8ms
Speed: 8.6ms preprocess, 167.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


['black-queen', 'black-rook', 'black-knight', 'black-pawn', 'white-king', 'white-queen', 'white-knight', 'white-knight', 'white-bishop', 'black-rook', 'black-king', 'white-pawn', 'black-pawn', 'black-bishop', 'white-rook', 'black-bishop', 'white-pawn', 'white-pawn']
tensor([0.9489, 0.8950, 0.8843, 0.8790, 0.8751, 0.8678, 0.8439, 0.8376, 0.8110, 0.8085, 0.8066, 0.7946, 0.7848, 0.7603, 0.7298, 0.6972, 0.5840, 0.4381])
1b4Q1/r2n1p2/4k3/3N4/1p1PB1bq/4N3/r5P1/4RK2 w
[['' 'b' '' '' '' '' 'Q' '']
 ['r' '' '' 'n' '' 'p' '' '']
 ['' '' '' '' 'k' '' '' '']
 ['' '' '' 'N' '' '' '' '']
 ['' 'p' '' 'P' 'B' '' 'b' 'q']
 ['' '' '' '' 'N' '' '' '']
 ['r' '' '' '' '' '' 'P' '']
 ['' '' '' '' 'R' 'K' '' '']]
